This notebook is a section to the protocol, *Use of scREAD to Explore and Analyze Single-cell and Single-nucleus RNA-Seq data for Alzheimer’s Disease*

If you have any questions or feedback regarding this notebook, please contact Cankun Wang <cankun.wang@osumc.edu>.

## Outline 

0. How to use this notebook
1. Background
2. Install dependencies and curate the data from scREAD
3. Specify parameters settings 
4. Interpret the result table

## 0. How to use this notebook
This notebook utilizes Google Colab , which is an interactive computational enviroment that combines live code, visualizations, and explanatory text. To run this notebook, you may first need to make a copy by choosing File > Save a Copy in Drive from the menu bar (may take a few moments to save).

The notebook is organized into a series of cells. You can modify the R command and execute each cell as you would a Jupyter notebook/R notebook. To run all of the cells at once, choose **Runtime > Run all** from the menu bar.

## 1. Background

The notebook provide table to mainly answer the question:

For all differentially expressed genes (DEGs) in Alzheimer’s Disease (AD) vs control datasets comparisons at a cell type of interest from scREAD, what genes are commanly ranked at top positions by the log-foldchanges? 

## 2. Install dependencies and curate the data from scREAD

First, let's install some necessary dependencies in this project, this should take about **10 minutes**:

In [1]:
install.packages('RVenn', repos='http://cran.rstudio.com/')
install.packages('rlist', repos='http://cran.rstudio.com/')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘tweenr’, ‘polyclip’, ‘RcppEigen’, ‘permute’, ‘ggforce’, ‘vegan’, ‘pheatmap’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘XML’, ‘data.table’




Next, we will load the R packages, scREAD data (~70MB), and pre-defined functions to calculate the overlapping genes:

In [12]:

library(tidyverse)
library(RVenn)
library(rlist)
library(knitr)

## Load data from GitHub or use the alternative server

tryCatch({
  load(url('https://raw.githubusercontent.com/OSU-BMBL/scread-protocol/master/overlapping_genes/scread_db.rdata'))
}, error= function(e) {
   message("Failed to load data from GitHub , trying alternative server...")
   load(url('https://bmbl.bmi.osumc.edu/downloadFiles/scread/protocol/scread_db.rdata'))
})
## The scread_db.rdata contains scREAD database DEGs data, a function named 'calc_overlap_list' to perform the analysis

Warning message in load(url("https://raw.githubusercontent.com/OSU-BMBL/scread-protocol/master/overlapping_genes/scread_db3.rdata")):
“cannot open URL 'https://raw.githubusercontent.com/OSU-BMBL/scread-protocol/master/overlapping_genes/scread_db3.rdata': HTTP status was '404 Not Found'”
Failed to load data from GitHub , trying alternative server...

Warning message in load(url("https://bmbl.bmi.osumc.edu/downloadFiles/scread/protocol/scread_db.rdata")):
“input string '10<96>12 months' cannot be translated to UTF-8, is it valid in 'CP936'?”


## 3. Specify parameters settings 

To calculate overlapping genes, these parameters are needed:

- The number of top genes in each AD vs control DE results (default=100)

- Species (default=Human)

- Brain region (e.g,Entorhinal Cortex)

- DE direction (e.g, up)

- Overlap threshold (for example, a gene is an overlapping gene if a should at least appeared 3 times in total 4 comparisons. Here the threshold=3)

By default two tables will be generated: 

1. The overlapping genes in the selected brain region

2. The detailed information, including rankings, log-foldchange, dataset source information from the overlapping genes


First, we can process some of our metadata:

In [14]:
REGION_LIST <- sort(unique(dataset$region))
CT_LIST <- sort(unique(cell_type_meta$cell_type))
CT_SHORT_LIST <- CT_LIST
CT_SHORT_LIST[CT_LIST=="Oligodendrocyte precursor cells"] <- "opc"
CT_SHORT_LIST <- tolower(substr(CT_SHORT_LIST, 1, 3))


We can take a look at what brain regions, cell types, and cell types abbrevations are included in scREAD:

In [15]:
list(brain_region=REGION_LIST, cell_type=CT_LIST, short_name=CT_SHORT_LIST)

$brain_region
 [1] "Cerebellum"                          "Cerebral cortex"                    
 [3] "Cortex"                              "Cortex and hippocampus"             
 [5] "Entorhinal Cortex"                   "Hippocampus"                        
 [7] "Prefrontal cortex"                   "Subventricular zone and hippocampus"
 [9] "Superior frontal gyrus (BA8)"        "Superior parietal lobe"             

$cell_type
[1] "Astrocytes"                      "Endothelial cells"              
[3] "Excitatory neurons"              "Inhibitory neurons"             
[5] "Microglia"                       "Oligodendrocyte precursor cells"
[7] "Oligodendrocytes"               

$short_name
[1] "ast" "end" "exc" "inh" "mic" "opc" "oli"

Below are the necessary settings in order to calculate the overlapping genes. **Feel free to change these parameters**!

In [16]:
# We use top 100 DE genes in each AD vs control comparison
TOP <- 100

# Species should be either 'Human' or 'Mouse'
this_species <- 'Human'

# Specify our brain region of interest, here we selected the 5th brain region in REGION_LIST variable, i.e, Entorhinal Cortex'
this_region <- REGION_LIST[5]

# DE direction should either 'up' or 'down', 'up' means we select DE genes that are expessed higher in the disease dataset (the first group)
this_direction <- 'up'

# The OVERLAP_THRES should be manually defined based on your interest and total number of comparisons in scREAD. 
# For example, scREAD have 4 total AD vs control datasets comparisons, we set the threshold to 3, meaning that we want to find overlapping genes that are at least appeared in 3 comparisons
OVERLAP_THRES <- 3

# Now, we can calculate the overlapping genes based on the parameters above, the results are stored in a list variable:
result <- calc_overlap_list()


## 4. Interpret the result table

### 1. Overlapping DEGs among cell types

If you are using the default settings, the second row in the table below can be interpreted as:

'For all AD vs control datasets comparisons in Human Entorhinal Cortex Astrocytes (ast), the *GFAP* gene ranked top 100 by log-foldchange values in at least 3 comparisons'.


In [17]:
kable(result$list)



|ct          |gene      |
|:-----------|:---------|
|ast,mic,oli |NEAT1     |
|ast         |ITGB4     |
|ast         |HSPA1B    |
|ast         |GFAP      |
|end         |IFITM3    |
|end         |IFITM2    |
|exc         |PLP1      |
|exc         |FTH1      |
|mic         |SPP1      |
|mic         |DPYD      |
|mic         |ACSL1     |
|oli         |LINC00486 |

### 2. The details of the overlapping gene from all cell types

The table below shows the details of the overlapping genes. 

In [18]:
kable(result$rank[,c(1:6,9)])



|ct  |gene      | avg_logFC|disease_id |control_id |rank | mean_rank|
|:---|:---------|---------:|:----------|:----------|:----|---------:|
|mic |ACSL1     |  1.295940|AD00203    |AD00201    |19   | 27.000000|
|mic |ACSL1     |  0.557061|AD00205    |AD00201    |52   | 27.000000|
|mic |ACSL1     |  1.650250|AD00206    |AD00201    |10   | 27.000000|
|mic |DPYD      |  1.822040|AD00203    |AD00201    |4    |  9.500000|
|mic |DPYD      |  1.536450|AD00204    |AD00202    |3    |  9.500000|
|mic |DPYD      |  0.910131|AD00205    |AD00201    |24   |  9.500000|
|mic |DPYD      |  1.188850|AD00206    |AD00201    |7    |  9.500000|
|exc |FTH1      |  1.749390|AD00203    |AD00201    |16   | 12.000000|
|exc |FTH1      |  0.673986|AD00205    |AD00201    |15   | 12.000000|
|exc |FTH1      |  1.201300|AD00206    |AD00201    |5    | 12.000000|
|ast |GFAP      |  1.217740|AD00203    |AD00201    |8    | 13.333333|
|ast |GFAP      |  1.080320|AD00204    |AD00202    |4    | 13.333333|
|ast |GFAP      | 

Take *GFAP* gene as an example, we know *GFAP* is an overlapping gene in Human Entorhinal Cortex Astrocytes. From the section of the table below, we know the *GFAP* ranked top 100 in 3 comparisons of 4 total comparisons, the mean rank of the gene is 13, and the average log-foldchanges of *GFAP* in each comparisons are also listed. 

In [19]:
#|ct  |gene      | avg_logFC|disease_id |control_id |rank | overlapping_comparison| total_comparison| mean_rank|
#|:---|:---------|---------:|:----------|:----------|:----|----------------------:|----------------:|---------:|
#|ast |GFAP      |  1.217740|AD00203    |AD00201    |8    |                      3|                4| 13.333333|
#|ast |GFAP      |  1.080320|AD00204    |AD00202    |4    |                      3|                4| 13.333333|
#|ast |GFAP      |  0.553044|AD00206    |AD00201    |28   |                      3|                4| 13.333333|